**Vehicle Detection and Tracking Project**

*Extending Advances Lane Finding Project @ https://github.com/Matz89/CarND-Advanced-Lane-Lines *

The goals / steps of this project are the following:

* Perform a Histogram of Oriented Gradients (HOG) feature extraction on a labeled training set of images and train a classifier Linear SVM classifier

* Optionally, you can also apply a color transform and append binned color features, as well as histograms of color, to your HOG feature vector.

* Note: for those first two steps don't forget to normalize your features and randomize a selection for training and testing.

* Implement a sliding-window technique and use your trained classifier to search for vehicles in images.

* Run your pipeline on a video stream (start with the test_video.mp4 and later implement on full project_video.mp4) and create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.

* Estimate a bounding box for vehicles detected.

**Import Packages**

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import glob

**Read and display all Test Images**  

--using cv2.imread

*Test Road Images*

In [ ]:
#image reader
test_img_path = ".\\test_images\\"
test_img_filename = '*'
test_images = glob.glob(test_img_path + test_img_filename)

#images contained here
img_shp = cv2.imread(test_images[0]).shape
test_imgs = np.empty((len(test_images), img_shp[0], img_shp[1], img_shp[2]), dtype="uint8")

#Display all images with matplotlib
figx = 2
figy = 4
#f, axarr = plt.subplots(figx,figy, figsize=(20,10))
y = 0
#for i, img in enumerate(test_images):
    #this_img = cv2.imread(img)
    #this_img_resized = cv2.resize(this_img,(img_shp[1], img_shp[0]))
    #test_imgs[i] = this_img_resized
    #x = i % figx
    #axarr[x,y].imshow(cv2.cvtColor(this_img, cv2.COLOR_BGR2RGB))
    #axarr[x,y].set_title(img.split('\\')[-1])

    #y = y+1 if x >= figx-1 else y

#Camera image details
img_w = img_shp[1]
img_h = img_shp[0]
img_ch = img_shp[2]
num_imgs = len(test_imgs)

print("Number of Images: {0}\nImage Width: {1}\nImage height: {2}\nColor Channels: {3}".format(num_imgs, img_w, img_h, img_ch))
  
#display images
#plt.tight_layout()
#plt.show()
    

*Camera Calibration Images*

In [ ]:
#image reader
camera_img_path = ".\\camera_cal\\"
camera_img_filename = '*'
camera_images = glob.glob(camera_img_path + camera_img_filename)

#images contained here - use shape of first image - fixed sized for greater performance
img_shp = cv2.imread(camera_images[0]).shape
camera_imgs = np.empty((len(camera_images), img_shp[0], img_shp[1], img_shp[2]), dtype="uint8")

#Display all images with matplotlib
figx = 5
figy = 4
#f, axarr = plt.subplots(figx,figy, figsize=(20,10))
y = 0
#for i, img in enumerate(camera_images):
    #this_img = cv2.imread(img)
    #this_img_resized = cv2.resize(this_img,(img_shp[1], img_shp[0]))
    #camera_imgs[i] = np.uint8(this_img_resized)
    #x = i % figx
    #axarr[x,y].imshow(cv2.cvtColor(this_img, cv2.COLOR_BGR2RGB))
    #axarr[x,y].set_title(img.split('\\')[-1])

    #y = y+1 if x >= figx-1 else y

#Camera image details
img_w = img_shp[1]
img_h = img_shp[0]
img_ch = img_shp[2]
num_imgs = len(camera_imgs)

print("Number of Images: {0}\nImage Width: {1}\nImage height: {2}\nColor Channels: {3}".format(num_imgs, img_w, img_h, img_ch))
    
#display images    
#plt.tight_layout()
#plt.show()



**Helper Functions**

*Camera Calibration Functions*

In [ ]:
#For use with chessboard calibration only!
#return retval, undist, cornersImg
def calibrate_camera(calibration_imgs, nx, ny):
    objp = np.zeros((nx*ny, 3), np.float32)
    objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2) #object x,y coordinates

    objpoints = []
    imgpoints = []
    
    for i, src_img in enumerate(camera_imgs):
        img = np.copy(src_img)
        
        #We assume BGR format
        #Convert to Grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        #Find chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

        if ret == True:
            #Draw and display the corners
            cornersImg = cv2.drawChessboardCorners(img, (nx, ny), corners, ret)

            objpoints.append(objp)
            imgpoints.append(corners)

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    return mtx, dist

    
#Example/Test  
mtx, dist = calibrate_camera(camera_imgs, 9, 6)

#for img in camera_imgs:
    
    #timg = np.copy(img)
    #undist = cv2.undistort(timg, mtx, dist, None, mtx)

    #f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    #f.tight_layout()
    #ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    #ax1.set_title('Original Image', fontsize=50)
    #ax2.imshow(cv2.cvtColor(undist, cv2.COLOR_BGR2RGB)) 
    #ax2.set_title("Undistorted Image", fontsize=50)
    #plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    

*Colour Transforms and Gradients*

In [ ]:
#Use Sobel and HLS thresholding to find edges in image
def findSobelxHLSThresh(src_img, s_thresh=(170,255), sx_thresh=(20,100), asColour=False):
    img = np.copy(src_img)
    
    #Covert to HLS color space and separate the channels
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    l_ch = hls[:,:,1]
    s_ch = hls[:,:,2]
    
    #Sobel x
    sobelx = cv2.Sobel(l_ch, cv2.CV_64F, 1, 0) #Take the derivative in x
    abs_sobelx = np.absolute(sobelx) #Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    #Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    
    #Threshold color channel
    s_binary = np.zeros_like(s_ch)
    s_binary[(s_ch >= s_thresh[0]) & (s_ch <= s_thresh[1])] = 1
  
    #Stack channels; Green = sobelx, Blue = Saturation
    if asColour:
        color_binary = np.dstack((np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    else:
        combined = np.add(sxbinary, s_binary)
        color_binary = np.dstack((combined, combined, combined))
        color_binary[color_binary > 0] = 255
    

    return color_binary
    
        

#Example/Test
#for i, img in enumerate(test_imgs):

    #edgeResult = findSobelxHLSThresh(img, asColour = True)
    #f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    #f.tight_layout()

    #ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    #ax1.set_title('Original Image', fontsize=40)

    #ax2.imshow(edgeResult)
    #ax2.set_title('Colour Result', fontsize=40)
    #plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)


*Perspective Transform*

In [ ]:
#Defined Region coordinates
TOP_LEFT_XY = [.44,.65]
TOP_RIGHT_XY = [.56,.65]
BOTTOM_LEFT_XY = [.20,.92]
BOTTOM_RIGHT_XY = [.80,.92]

#overlays image with polygon defined by vertices
def view_region_of_interest(img, vertices):
    r_img = np.copy(img)
    vertices = vertices[0]
    prev = vertices[-1]

    for v in vertices:
        #creating a polygon defined by "vertices" with the fill color Blue   
        cv2.line(r_img,(prev[0], prev[1]),(v[0],v[1]),(255,0,0),5)
        prev = v

    return r_img


#Example/Test
tImage = test_imgs[0]

#Define Region of Interest (x,y)
v1 = np.multiply(np.flip(tImage.shape[:2], axis=0), BOTTOM_LEFT_XY)  #Bottom Left
v2 = np.multiply(np.flip(tImage.shape[:2], axis=0), TOP_LEFT_XY)     #Top Left 
v3 = np.multiply(np.flip(tImage.shape[:2], axis=0), TOP_RIGHT_XY)    #Top Right
v4 = np.multiply(np.flip(tImage.shape[:2], axis=0), BOTTOM_RIGHT_XY) #Bottom Right


verts = np.array([v1,v2,v3,v4], dtype=np.int32)

region_image = view_region_of_interest(tImage, [verts])

#f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
#f.tight_layout()

#ax1.imshow(cv2.cvtColor(tImage, cv2.COLOR_BGR2RGB))
#ax1.set_title('Original Image', fontsize=40)

#ax2.imshow(cv2.cvtColor(region_image, cv2.COLOR_BGR2RGB))
#ax2.set_title('Region Result', fontsize=40)
#plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
#Offsets for perspective transform
w_offset = 300
h_offset = 50

def topDownPerspective(img, src_pts, dst_pts):
    pImg = np.copy(img)
    img_size = (pImg.shape[1], pImg.shape[0])
    
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)

    warped = cv2.warpPerspective(pImg, M, img_size, flags=cv2.INTER_LINEAR)
    return warped, M

#Example/Test
for i, img in enumerate(test_imgs):
    
    shpy = img.shape[0]
    shpx = img.shape[1]
    img_size = (shpx, shpy)
    
    src_pts = np.float32(verts)
    
    dst_tl = [w_offset,h_offset]
    dst_tr = [shpx-w_offset,h_offset]
    dst_bl = [w_offset,shpy - h_offset]
    dst_br = [shpx-w_offset,shpy-h_offset]
    
    dst_pts = np.float32([dst_bl, dst_tl, dst_tr, dst_br])
    undist_img = cv2.undistort(img, mtx, dist, None, mtx)
    edge_img = findSobelxHLSThresh(undist_img, asColour=False)
    result_img, result_M = topDownPerspective(edge_img,src_pts, dst_pts )

    #f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    #f.tight_layout()

    #ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    #ax1.set_title('Original Image', fontsize=40)

    #ax2.imshow(result_img, cmap='gray')
    #ax2.set_title('TopDown Result', fontsize=40)
    #plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

*Lane Finding*
   

In [ ]:
def identifyLanes(src_img, nwindows=9, margin=100, minpix=50):
    #nwindowsl Choose the number of sliding windows
    #margin; Set the width of the windows +/- margin
    #minpix; Set the minimum number of pixels found to recenter window
    
    #Prep copy of image
    img = np.copy(src_img)
    img = np.sum(img, axis=-1)
    img[img > 0] = 255
    
    #collection of rectangle tuples
    rectangles = []
    
    # Take a histogram of the bottom half of the image
    histogram = np.sum(img[img.shape[0]//2:,:], axis=0)
    
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((img,img,img))
    out_img = out_img.astype("uint8")
    
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    # Set height of windows
    window_height = np.int(img.shape[0]//nwindows)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        #Pack away rectangle points to show areas of interest - can be drawn later
        rectangles.append(((win_xleft_low,win_y_low),(win_xleft_high,win_y_high)))
        rectangles.append(((win_xright_low,win_y_low),(win_xright_high,win_y_high)))

        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
    
    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
    
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    #Colouring lanes left=Red, right=Blue
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    
    #Generating x and y values for plotting
    ploty = np.linspace(0, out_img.shape[0]-1, out_img.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return out_img, left_fit, right_fit, rectangles, (left_fitx, right_fitx, ploty)
    
    
    
#Example/Test
for i, img in enumerate(test_imgs):
    
    shpy = img.shape[0]
    shpx = img.shape[1]
    img_size = (shpx, shpy)
    
    src_pts = np.float32(verts)
    
    dst_tl = [w_offset,h_offset]
    dst_tr = [shpx-w_offset,h_offset]
    dst_bl = [w_offset,shpy - h_offset]
    dst_br = [shpx-w_offset,shpy-h_offset]
    
    dst_pts = np.float32([dst_bl, dst_tl, dst_tr, dst_br])
    undist_img = cv2.undistort(img, mtx, dist, None, mtx)
    edge_img = findSobelxHLSThresh(undist_img, asColour=False)
    topdown_img, topdown_M = topDownPerspective(edge_img,src_pts, dst_pts )
    result_img, left_fit, right_fit, rects, img_plots = identifyLanes(topdown_img)
    
    ploty = img_plots[2]
    left_fitx = img_plots[0]
    right_fitx = img_plots[1]
    
    #Draw Rectangles for output image
    for pts in rects:
        cv2.rectangle(result_img,pts[0],pts[1],(0,255,0), 2) 
    
    #f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    #f.tight_layout()

    #ax1.imshow(topdown_img, cmap='gray')
    #ax1.set_title('TopDown View', fontsize=40)
    
    #ax2.imshow(result_img)
    #ax2.set_title('Identified Lanes', fontsize=40)
    #ax2.plot(left_fitx, ploty, color='yellow')
    #ax2.plot(right_fitx, ploty, color='yellow')
    #plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
def identifyLanesNext(src_img, left_fit, right_fit, margin=100):
    #If providing left_fit/right_fit polynomials (ie after finding the lanes in the previous frame with identifyLanes)
    #This will find new points within the margin of the left_fit/right_fit lines
    out_img = np.copy(src_img)
    nonzero = out_img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
    left_fit[1]*nonzeroy + left_fit[2] + margin))) 

    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
    right_fit[1]*nonzeroy + right_fit[2] + margin)))  
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, out_img.shape[0]-1, out_img.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    #Colouring lanes left=Red, right=Blue
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    
    return out_img, left_fit, right_fit, (left_fitx, right_fitx, ploty)

*Measuring Curvature*

In [ ]:
def findCurvatureOfLane(ploty, leftx, rightx):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Define y-value where we want radius of curvature
    # I'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return left_curverad, right_curverad

In [ ]:
def findDevFromCenter(ploty, leftfit, rightfit, car_pos_x, img_h):
    # Define conversions in x and y from pixels space to meters
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    left_x_val = leftfit[0]*img_h**2 + leftfit[1]*img_h + leftfit[2]
    right_x_val = rightfit[0]*img_h**2 + rightfit[1]*img_h + rightfit[2]
    
    #Lane Center position
    lane_center_x = (left_x_val + right_x_val) / 2
    car_deviation = (car_pos_x - lane_center_x) * xm_per_pix
    
    return car_deviation


**Vehicle Detection and Tracking**

*Reading in Vehicle and Non-Vehicle Images for Training/Testing*

In [ ]:
#image reader
training_img_path = ".\\training\\"
non_vehicle_img_filenames = 'non-vehicles\\*\\*.png'
vehicle_img_filenames = 'vehicles\\*\\*.png'
non_vehicle_images = glob.glob(training_img_path + non_vehicle_img_filenames)
vehicle_images = glob.glob(training_img_path + vehicle_img_filenames)

#images contained here
img_shp = cv2.imread(non_vehicle_images[0]).shape
non_vehicle_imgs = np.empty((len(non_vehicle_images), img_shp[0], img_shp[1], img_shp[2]), dtype="uint8")

#Display random images with matplotlib
figx = 2
figy = 4
f, axarr = plt.subplots(figx,figy, figsize=(20,10))
y = 0
for i, veh_img in enumerate(vehicle_images):
    this_veh_img = cv2.imread(veh_img)
    this_non_veh_img = cv2.imread(non_vehicle_images[i])
    x = i % figx
    axarr[0,y].imshow(cv2.cvtColor(this_veh_img, cv2.COLOR_BGR2RGB))
    axarr[0,y].set_title("Vehicle Image")
    axarr[1,y].imshow(cv2.cvtColor(this_non_veh_img, cv2.COLOR_BGR2RGB))
    axarr[1,y].set_title("Non-Vehicle Image")

    y = y+1 if x >= figx-1 else y
    if y>=figy:
        break

#Training image details
num_veh_imgs = len(vehicle_images)
num_nonveh_imgs = len(non_vehicle_images)

print("Number of Vehicle Images: {0}\nNumber of Non-Vehicle Images: {1}".format(num_veh_imgs, num_nonveh_imgs))
  
#display images
plt.tight_layout()
plt.show()
    

*Extracting Histogram of Oriented Gradients (HOG) *

In [ ]:
from skimage.feature import hog

def get_hog_features(img, orient=9, pix_per_cell=8, cell_per_block=2, vis=True, feature_vec=True):
                         
    return_list = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block),
                                  block_norm= 'L2-Hys', transform_sqrt=False, 
                                  visualise= vis, feature_vector= feature_vec)
    
    # name returns explicitly
    hog_features = return_list[0]
    if vis:
        hog_image = return_list[1]
        return hog_features, hog_image
    else:
        return hog_features
    
#Example/Test
#Let random index for vehicle image/Non-Vehicle image
v_ind = np.random.randint(0, num_veh_imgs)
nv_ind = np.random.randint(0, num_nonveh_imgs)

#Read Images
v_img = cv2.imread(vehicle_images[v_ind])
nv_img = cv2.imread(non_vehicle_images[nv_ind])

#Get Grayscale of Images
v_gray = cv2.cvtColor(v_img, cv2.COLOR_BGR2GRAY)
nv_gray = cv2.cvtColor(nv_img, cv2.COLOR_BGR2GRAY)

# Call our function with vis=True to see an image output
v_features, v_hog_image = get_hog_features(v_gray, orient= 9, 
                        pix_per_cell= 8, cell_per_block= 2, 
                        vis=True, feature_vec=False)

nv_features, nv_hog_image = get_hog_features(nv_gray, orient= 9, 
                        pix_per_cell= 8, cell_per_block= 2, 
                        vis=True, feature_vec=False)

f, axarr = plt.subplots(2,2, figsize=(20,10))
axarr[0,0].imshow(v_gray, cmap='gray')
axarr[0,0].set_title("Vehicle Image")
axarr[0,1].imshow(nv_gray, cmap='gray')
axarr[0,1].set_title("Non-Vehicle Image")
axarr[1,0].imshow(v_hog_image, cmap='gray')
axarr[1,0].set_title("HOG Visualization (Vehicle)")
axarr[1,1].imshow(nv_hog_image, cmap='gray')
axarr[1,1].set_title("HOG Visualization (Non-Vehicle)")



*Extracting Colour Histogram Features*

In [ ]:
def color_hist(img, nbins=32, bins_range=(0, 256)):
    
    #Separate colour channels and find histograms
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    
    #Concat into single vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    
    return hist_features

#Example/Test
#Let random index for vehicle image/Non-Vehicle image
v_ind = np.random.randint(0, num_veh_imgs)
nv_ind = np.random.randint(0, num_nonveh_imgs)

#Read Images
v_img = cv2.imread(vehicle_images[v_ind])
nv_img = cv2.imread(non_vehicle_images[nv_ind])

#Colour Histograms
v_hists = color_hist(v_img)
nv_hists = color_hist(nv_img)

v_rh = v_hists[:32]
v_gh = v_hists[32:64]
v_bh = v_hists[64:]

nv_rh = nv_hists[:32]
nv_gh = nv_hists[32:64]
nv_bh = nv_hists[64:]

bin_edges = np.histogram(v_img[:,:,0], bins=32, range=(0,256))[1]
bin_centers = (bin_edges[1:]  + bin_edges[0:len(bin_edges)-1])/2

fig = plt.figure(figsize=(12,3))
plt.subplot(141)
plt.imshow(cv2.cvtColor(v_img, cv2.COLOR_BGR2RGB))
plt.title("IMAGE")
plt.subplot(142)
plt.bar(bin_centers, v_rh)
plt.xlim(0, 256)
plt.title('R Histogram')
plt.subplot(143)
plt.bar(bin_centers, v_gh)
plt.xlim(0, 256)
plt.title('G Histogram')
plt.subplot(144)
plt.bar(bin_centers, v_bh)
plt.xlim(0, 256)
plt.title('B Histogram')
fig.tight_layout()

fig = plt.figure(figsize=(12,3))
plt.subplot(141)
plt.imshow(cv2.cvtColor(nv_img, cv2.COLOR_BGR2RGB))
plt.title("IMAGE")
plt.subplot(142)
plt.bar(bin_centers, nv_rh)
plt.xlim(0, 256)
plt.title('R Histogram')
plt.subplot(143)
plt.bar(bin_centers, nv_gh)
plt.xlim(0, 256)
plt.title('G Histogram')
plt.subplot(144)
plt.bar(bin_centers, nv_bh)
plt.xlim(0, 256)
plt.title('B Histogram')
fig.tight_layout()

*Extracting Spatial Bin Features*

In [ ]:
def bin_spatial(img, size=(32, 32)):
    #Use Ravel to make image 1D
    features = cv2.resize(img, size).ravel() 
    
    return features

#Examples/Test
#Let random index for vehicle image/Non-Vehicle image
v_ind = np.random.randint(0, num_veh_imgs)
nv_ind = np.random.randint(0, num_nonveh_imgs)

#Read Images
v_img = cv2.imread(vehicle_images[v_ind])
nv_img = cv2.imread(non_vehicle_images[nv_ind])

#extract spatial feature
v_spatial = bin_spatial(v_img)
nv_spatial = bin_spatial(nv_img)

#plot for visual
fig = plt.figure(figsize=(20,4))
plt.subplot(121)
plt.plot(v_spatial)
plt.title("Vehicle Spatial Features")
plt.subplot(122)
plt.plot(nv_spatial)
plt.title("Non-Vehicle Spatial Features")
fig.tight_layout()

*Extract features of a multiple images*

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler

def extract_features(img_list, color_space='BGR', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):   
    #Contain all features
    features = []
    
    for file in img_list:
        print(file)
        #Init empty list to contain image features
        img_features = []
        
        img = cv2.imread(file)

        #Colour Conversion
        if color_space != 'BGR':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
            elif color_space == 'RGB':
                feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        else: feature_image = np.copy(img)      

        #Spatial Features
        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            img_features.append(spatial_features)

        #Histogram Features
        if hist_feat == True:
            hist_features = color_hist(feature_image, nbins=hist_bins)
            img_features.append(hist_features)

        #HOG Features
        if hog_feat == True:
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))      
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            img_features.append(hog_features)
        features.append(np.concatenate(img_features))
    return features

*Train the model*

In [ ]:
#TUNE-ABLE PARAMETERS
color_space = 'HLS'       # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9                # HOG orientations
pix_per_cell = 8          # HOG pixels per cell
cell_per_block = 2        # HOG cells per block
hog_channel = 2           # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16)   # Spatial binning dimensions
hist_bins = 16            # Number of histogram bins
spatial_feat = True       # Spatial features on or off
hist_feat = False         # Histogram features on or off
hog_feat = True           # HOG features on or off
y_start_stop = [450, 700] # Min and max in y to search in slide_window()

In [ ]:
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split

#Sample to test home pc
sample_size = 500
cars = vehicle_images[0:sample_size]
notcars = non_vehicle_images[0:sample_size]

vehicle_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

non_vehicle_features = extract_features(notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

#Stack features
X = np.vstack((vehicle_features, non_vehicle_features)).astype(np.float64)

#Init labels and stack same order as above
y = np.hstack((np.ones(len(vehicle_features)), np.zeros(len(non_vehicle_features))))

#Split into training and test sets (random)
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rand_state)

# Fit a per-column scaler
X_scaler = StandardScaler().fit(X_train)
# Apply the scaler to X
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

print('Using:',orient,'orientations',pix_per_cell,'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()

*Window Search*

In [ ]:
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):

    #Init positive detection window list
    on_windows = []

    #Find positives
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows

**Main Pipeline to Find Lanes**

In [ ]:
#REDEFINE Region Coordinates
TOP_LEFT_XY = [.44,.65]
TOP_RIGHT_XY = [.56,.65]
BOTTOM_LEFT_XY = [.20,.92]
BOTTOM_RIGHT_XY = [.80,.92]

#Offsets for perspective transform
w_offset = 300
h_offset = 50

#Camera Calibrations
mtx, dist = calibrate_camera(camera_imgs, 9, 6)

#Main pipeline to find lanes and return augmented image displaying lane information
class showLane: 
    def __init__(self):
        self.left_fit = None
        self.right_fit = None
    def __call__(self, src_img):
        #do not edit original image
        img = np.copy(src_img)

        line_width=50

        #Image shape information
        shpy = img.shape[0]
        shpx = img.shape[1]
        img_size = (shpx, shpy)

        #Source points for perspective Transform (And region of interest for Lanes)
        v1 = np.multiply(np.flip(img.shape[:2], axis=0), BOTTOM_LEFT_XY)  #Bottom Left
        v2 = np.multiply(np.flip(img.shape[:2], axis=0), TOP_LEFT_XY)     #Top Left 
        v3 = np.multiply(np.flip(img.shape[:2], axis=0), TOP_RIGHT_XY)    #Top Right
        v4 = np.multiply(np.flip(img.shape[:2], axis=0), BOTTOM_RIGHT_XY) #Bottom Right

        src_pts = np.float32(verts)

        #Destination points for perspective transform
        dst_tl = [w_offset,h_offset]
        dst_tr = [shpx-w_offset,h_offset]
        dst_bl = [w_offset,shpy - h_offset]
        dst_br = [shpx-w_offset,shpy-h_offset]

        dst_pts = np.float32([dst_bl, dst_tl, dst_tr, dst_br])

        #Undistort image
        undist_img = cv2.undistort(img, mtx, dist, None, mtx)

        #Push image through threshhold filtering
        thresh_img = findSobelxHLSThresh(undist_img, asColour=False)

        #Create Top-Down perspective of relevant area of image
        topdown_img, topdown_M = topDownPerspective(thresh_img,src_pts, dst_pts )
        
        if(self.left_fit != None) and (self.right_fit != None):
            fit_img, left_fit, right_fit, img_plots = identifyLanesNext(topdown_img, self.left_fit, self.right_fit)
            self.left_fit = left_fit
            self.right_fit = right_fit
        else:
            fit_img, left_fit, right_fit, rects, img_plots = identifyLanes(topdown_img)
            self.left_fit = left_fit
            self.right_fit = right_fit

        #separate out img_plots
        ploty = img_plots[2]
        left_fitx = img_plots[0]
        right_fitx = img_plots[1]

        #Find curvature of lanes; radius in meters
        left_curverad, right_curverad = findCurvatureOfLane(ploty, left_fitx, right_fitx)
        
        #Find vehicle deviation from center of lane
        car_dev = findDevFromCenter(ploty, self.left_fit, self.right_fit, img.shape[1]/2, img.shape[0])

        #Create overlay image
        out_img_faded = np.zeros_like(fit_img) 
        out_img = np.zeros_like(fit_img) 

        #Create Lane Line Visual Info    
        #True line width
        line_width = int(line_width / 2)

        #recast the x and y points into usable format for cv2.fillPoly()
        left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-line_width, ploty]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+line_width, ploty])))])
        left_line_pts = np.hstack((left_line_window1, left_line_window2))

        right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-line_width, ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+line_width, ploty])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))

        #Create Lane Region Visual Info
        #region_pts = np.concatenate((left_line_pts, np.flipud(right_line_pts)))
        region_left = np.array(np.transpose(np.vstack([left_fitx, ploty])))
        region_right = np.array(np.transpose(np.vstack([right_fitx, ploty])))
        region_pts = np.concatenate((region_left, np.flipud(region_right)))

        #Create Lane Curvature Visual Info (Average of both lanes)
        curve = np.array(np.transpose(np.vstack([(left_fitx+right_fitx)/2, ploty]))).astype("int")

        #Find a point along the line (near the bottom of the image
        curve_point = curve[-1]
        text_offset = 25;
        text_pos = (curve_point[0]+text_offset, curve_point[1]-text_offset)
        mid_curve_r = (left_curverad + right_curverad)/2

        #Draw Visuals
        region_colour = (0,255,0)
        left_lane_colour = (255,0,0)
        right_lane_colour = (0,0,255)
        curve_colour = (155,255,255)
        text_colour = (55,55,55)

        cv2.fillPoly(out_img_faded, [region_pts.astype('int32')], region_colour)
        cv2.fillPoly(out_img_faded, np.int_([left_line_pts]), left_lane_colour)
        cv2.fillPoly(out_img_faded, np.int_([right_line_pts]), right_lane_colour)

        start_pt = curve[0]
        for end_pt in curve[1:]:
            cv2.line(out_img, (start_pt[0], start_pt[1]), (end_pt[0], end_pt[1]), text_colour, 5)
            start_pt = end_pt

        #Return perspective to original view
        persp_img_faded, out_M = topDownPerspective(out_img_faded,dst_pts, src_pts )
        persp_img, out_M = topDownPerspective(out_img,dst_pts, src_pts )

        #Overlay with original image
        persp_img = cv2.addWeighted(persp_img_faded, 0.5, persp_img, 1.0, 0.0)
        out_img = cv2.addWeighted(persp_img, 1.0, undist_img, 1.0, 0.0)

        #Drawn interface information
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(out_img, "%.2fm (r.)" % mid_curve_r, text_pos, font, 1.0, curve_colour, 5)
        cv2.putText(out_img, "dev from cntr: %.2fm" % car_dev, (15, 45), font, 1.0, curve_colour, 5)

        return out_img

#Example/Test
#Image Test
t_img = test_imgs[4]

tt = showLane()
r_img = tt(t_img)


#Plot Example Image
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()

ax1.imshow(cv2.cvtColor(t_img, cv2.COLOR_BGR2RGB))
ax1.set_title('TopDown View', fontsize=40)

ax2.imshow(cv2.cvtColor(r_img, cv2.COLOR_BGR2RGB))
ax2.set_title('Pipeline Test', fontsize=40)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)


    

*Run pipeline with easy video sample*

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
#inp_vid = "./project_video.mp4"
#out_vid = "./output_videos/out_project_video.mp4"
#clip1 = VideoFileClip(inp_vid)
#out_clip = clip1.fl_image(showLane())
#out_clip.write_videofile(out_vid, audio=False)